In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from utils.vis_utils import load_biomarkers
import h5py
from utils.vis_utils import load_cell_meta
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import random

from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline

import time
from scipy.stats import spearmanr

from matplotlib import rcParams

In [ ]:

rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["Arial", "Liberation Sans", "DejaVu Sans"]

In [ ]:

random.seed(42)
np.random.seed(42)

## Correlation Analysis

### Pairwise correlation

In [ ]:
NUMBER_OF_IMAGES = 100
NUMBER_OF_SAMPLES = 20_000

In [ ]:
biomarkers=load_biomarkers()
biomarker_names = biomarkers.Antibody

#### Pixel level

In [ ]:

with h5py.File("data.h5", "r") as hf:
    vol_grp = hf["volumes"]
    data = vol_grp['data']
    training_idx = vol_grp["train_idx"][:NUMBER_OF_IMAGES]
    
    pixel_list = []
    for img_idx in training_idx:
        img_data = data[img_idx]  # shape: H x W x C
        pixels = img_data.reshape(-1, img_data.shape[-1])  # shape: (H*W, C)
        pixel_idxs = np.random.choice(pixels.shape[0], NUMBER_OF_SAMPLES, replace=False)
        pixels = pixels[pixel_idxs]
        pixel_list.append(pixels)
    
    all_pixels = np.vstack(pixel_list)
    log_pixels = np.log1p(all_pixels)

In [ ]:
variances = np.var(all_pixels, axis=0)
print("Channel variances:", variances)

##### Linear data

In [ ]:
def plot_corr_matrix(data, title, filename=None):
    pixel_corr_matrix = np.corrcoef(data, rowvar=False)

    fig = plt.figure(figsize=(18, 18))
    sns.heatmap(
        pixel_corr_matrix,
        xticklabels=biomarker_names,
        yticklabels=biomarker_names,
        cmap="coolwarm",
        center=0,
        square=True,
        cbar_kws={"shrink": 0.5, "label": "Pearson r", "orientation": "horizontal"},
    )
    plt.title(title, fontsize=24, pad=20)
    plt.tight_layout()
    plt.savefig(f"plots/{filename if filename else title}.pdf")
    plt.show()
    return pixel_corr_matrix

In [ ]:
nerve_biomarkers = [
    "VAChT",
    "MAP2",
    "Sox2",
    "TH",
    "NCAM",
    "Nestin",
    "Peripherin",
    "DCX",
    "Neurofilament",
]
def plot_peripherin_correlations(data, title, filename):
    peripherin_idx = None
    for i, name in enumerate(biomarker_names):
        if 'Peripherin' in name:
            peripherin_idx = i
            break

    if peripherin_idx is not None:
        peripherin_correlations = data[peripherin_idx, :]
        
        # Remove Peripherin's self-correlation for the plot
        plot_correlations = np.delete(peripherin_correlations, peripherin_idx)
        plot_biomarker_names = [name for i, name in enumerate(biomarker_names) if i != peripherin_idx]
        
        # Sort by correlation values (largest to smallest)
        sorted_indices = np.argsort(plot_correlations)
        plot_correlations_sorted = plot_correlations[sorted_indices]
        plot_biomarker_names_sorted = [plot_biomarker_names[i] for i in sorted_indices]
        
        colors = []
        for name in plot_biomarker_names_sorted:
            if name in nerve_biomarkers:
                colors.append('#FF6B6B')  # Red for nerve biomarkers
            else:
                colors.append('#4ECDC4')  # Teal for other biomarkers

        # Create a compact horizontal bar plot optimized for LaTeX
        plt.figure(figsize=(6, 8))  # Reduced width and height
        
        bars = plt.barh(range(len(plot_biomarker_names_sorted)), plot_correlations_sorted, color=colors,
                        alpha=0.8, edgecolor='black', linewidth=0.3)
            
        plt.ylabel('Biomarkers', fontsize=10, fontweight='bold')
        plt.xlabel('Correlation with Peripherin', fontsize=10, fontweight='bold')
        plt.title(title, fontsize=11, fontweight='bold', pad=15)
        
        # Smaller font sizes and tighter spacing
        plt.yticks(range(len(plot_biomarker_names_sorted)), plot_biomarker_names_sorted, fontsize=8)
        plt.xticks(fontsize=9)
        
        # Add vertical reference lines
        plt.axvline(x=0.05, color='gray', linestyle='--', alpha=0.5, linewidth=0.8)
        plt.axvline(x=0.1, color='gray', linestyle='--', alpha=0.5, linewidth=0.8)
        plt.axvline(x=0.2, color='gray', linestyle='--', alpha=0.5, linewidth=0.8)
        
        from matplotlib.patches import Patch
        legend_elements = [Patch(facecolor='#FF6B6B', label='Nerve Biomarkers'),
                        Patch(facecolor='#4ECDC4', label='Other Biomarkers')]
        plt.legend(handles=legend_elements, loc='lower right', fontsize=9)
        
        plt.grid(axis='x', alpha=0.3, linewidth=0.6)
        
        # Tighter layout with smaller margins
        plt.tight_layout(pad=1.0)
        
        # Save as PDF for LaTeX with high DPI and tight bounding box
        plt.savefig(f"plots/{filename}.pdf", dpi=300, bbox_inches='tight', 
                   facecolor='white', edgecolor='none')
        plt.show()

In [ ]:
pix_corr = plot_corr_matrix(all_pixels, "Pixel-level Correlation Matrix", "lin_pix_corr")

##### Log transformed

In [ ]:
log_corr = plot_corr_matrix(log_pixels, "Log transformed pixel level correlation matrix", "log_pix_corr")

In [ ]:

plot_peripherin_correlations(pix_corr, "Pixel-Wise peripherin correlations", "lin_pixel_peripherin_corr")

In [ ]:
plot_peripherin_correlations(log_corr, "Pixel-Wise peripherin correlations (log transformed)", "log_pixel_peripherin_corr")

#### Cell Level

In [ ]:
number_of_cells_per_image = 400
all_cell_means = []
all_cell_log_means = []
cell_ids = []
with h5py.File("data.h5", "r") as hf:
    vol_grp = hf["volumes"]
    data = vol_grp['data']  
    training_idx = vol_grp["train_idx"]
    # N*H*W*C

    case_ids = vol_grp["case_ids"]

    cell_mask_grp = hf["cell_masks"]
    cell_masks = cell_mask_grp["data"]
    # N*H*W
    
    # Get dimensions
    _, H, W, C = data.shape

    for train_idx in tqdm(range(min(len(training_idx),NUMBER_OF_IMAGES)), position=0):

        img_idx = training_idx[train_idx]


        # Get the current image slice and its cell mask
        img_data = data[img_idx]  # Shape: H*W*C
        img_mask = cell_masks[img_idx]  # Shape: H*W
        case_id = case_ids[img_idx]
        
        # Find unique cell IDs in this image (excluding 0 for background)
        img_cell_ids = np.unique(img_mask)
        img_cell_ids = img_cell_ids[img_cell_ids > 0]
        samples = np.random.choice(img_cell_ids, size=min(number_of_cells_per_image, len(img_cell_ids)), replace=False)

        for i, cell_id in enumerate(samples):
            print(f"processing cell {i}", end="\r")
            # Create mask for this specific cell in this specific image
            cell_mask = (img_mask == cell_id)  # Shape: H*W

            
            # Sum up biomarker values for this cell
            cell_means = np.zeros(C)
            cell_log_means = np.zeros(C)
            for c in range(C):
                # Apply mask to this channel and calculate mean
                cell_pixels = img_data[:, :, c][cell_mask]
                cell_log_pixels = np.log1p(img_data[:, :, c][cell_mask])
                cell_means[c] = np.mean(cell_pixels) if cell_pixels.size > 0 else 0
                cell_log_means[c] = np.mean(cell_log_pixels) if cell_log_pixels.size > 0 else 0
            
            all_cell_means.append(cell_means)
            all_cell_log_means.append(cell_log_means)
            cell_ids.append((case_id, cell_id))
        print()
            

In [ ]:
all_cell_log_means = np.array(all_cell_log_means)

In [ ]:
cell_ids = list(map(lambda x: (x[0].decode(), x[1]), cell_ids))
len(cell_ids)

##### Try to account for cell size bias

In [ ]:
all_cell_means = np.array(all_cell_means)

# Use DNA2 (index 37) as proxy for nuclear volume/total ion count
dna2_idx = 37
nuclear_volumes = all_cell_means[:, dna2_idx]

# Avoid division by zero by adding small epsilon
epsilon = 1e-6
nuclear_volumes_safe = nuclear_volumes + epsilon

# Normalize each cell's intensity vector by its nuclear volume
normalized_cell_means = all_cell_means / nuclear_volumes_safe[:, np.newaxis]

# Apply log transformation
log_normalized_cell_means = np.log1p(normalized_cell_means)

print(f"Original shape: {all_cell_means.shape}")
print(f"Normalized shape: {normalized_cell_means.shape}")
print(f"Log-normalized shape: {log_normalized_cell_means.shape}")
print(f"Nuclear volume range: {nuclear_volumes.min():.2f} - {nuclear_volumes.max():.2f}")

#### Only positive peripherin cells:

In [ ]:
# Find the index of Peripherin in biomarker_names
peripherin_idx = None
for i, name in enumerate(biomarker_names):
    if 'Peripherin' in name:
        peripherin_idx = i
        break

if peripherin_idx is not None:
    # Get peripherin values for all cells
    peripherin_values = all_cell_means[:, peripherin_idx]
    
    # Count positive values (greater than 0)
    positive_count = np.sum(peripherin_values > 0)
    total_count = len(peripherin_values)
    
    print(f"Peripherin positive cells: {positive_count}")
    print(f"Total cells: {total_count}")
    print(f"Percentage positive: {positive_count/total_count*100:.2f}%")
    
    # Show distribution of peripherin values
    print(f"Peripherin value range: {peripherin_values.min():.3f} - {peripherin_values.max():.3f}")
    print(f"Mean peripherin value: {peripherin_values.mean():.3f}")
    print(f"Median peripherin value: {np.median(peripherin_values):.3f}")
else:
    print("Peripherin not found in biomarker names")

##### Linear data

In [ ]:
cell_corr_matrix = plot_corr_matrix(all_cell_means, "Cell-aggregated raw correlation matrix", "lin_cell_corr")

### Nerve biomarkers
VAChT,
MAP2,
Sox2,
TH,
NCAM,
Peripherin,
DCX,
Neurofilament,

In [ ]:
# Find the index of Peripherin in biomarker_names
peripherin_idx = None
for i, name in enumerate(biomarker_names):
    if 'Peripherin' in name:
        peripherin_idx = i
        break

if peripherin_idx is not None:
    # Extract correlations between Peripherin and all other biomarkers
    peripherin_correlations = cell_corr_matrix[peripherin_idx, :]
    
    # Remove Peripherin's self-correlation for the plot
    plot_correlations = np.delete(peripherin_correlations, peripherin_idx)
    plot_biomarker_names = [name for i, name in enumerate(biomarker_names) if i != peripherin_idx]
    
    # Sort by correlation values (largest to smallest)
    sorted_indices = np.argsort(plot_correlations)[::-1]
    plot_correlations_sorted = plot_correlations[sorted_indices]
    plot_biomarker_names_sorted = [plot_biomarker_names[i] for i in sorted_indices]
    
    # Create a bar plot with improved visual design
    plt.figure(figsize=(16, 10))
    
    bars = plt.bar(range(len(plot_biomarker_names_sorted)), plot_correlations_sorted, 
                   alpha=0.8, edgecolor='black', linewidth=0.5)
    
    plt.xlabel('Biomarkers', fontsize=16, fontweight='bold')
    plt.ylabel('Correlation with Peripherin', fontsize=16, fontweight='bold')
    plt.title('Cell-level Correlations: Peripherin vs All Other Biomarkers', fontsize=18, fontweight='bold', pad=20)
    plt.xticks(range(len(plot_biomarker_names_sorted)), plot_biomarker_names_sorted, rotation=45, ha='right', fontsize=12)
    plt.yticks(fontsize=14)
    
    # Add horizontal reference lines
    plt.axhline(y=0.05, color='gray', linestyle='--', alpha=0.5, linewidth=1)
    plt.axhline(y=0.1, color='gray', linestyle='--', alpha=0.5, linewidth=1)
    plt.axhline(y=0.2, color='gray', linestyle='--', alpha=0.5, linewidth=1)
    
    plt.grid(axis='y', alpha=0.3, linewidth=0.8)
    plt.tight_layout()
    
    # Save as PDF for LaTeX with high DPI
    plt.savefig("plots/peripherin_correlations.pdf", dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print top positive and negative correlations (excluding self-correlation)
    sorted_indices = np.argsort(peripherin_correlations)[::-1]
    # Filter out the peripherin index from sorted results
    filtered_indices = [i for i in sorted_indices if i != peripherin_idx]
    
    print("Top 5 positive correlations with Peripherin:")
    for i in filtered_indices[:5]:
        print(f"{biomarker_names[i]}: {peripherin_correlations[i]:.3f}")
    
    print("\nTop 5 negative correlations with Peripherin:")
    for i in filtered_indices[-5:]:
        print(f"{biomarker_names[i]}: {peripherin_correlations[i]:.3f}")
else:
    print("Peripherin not found in biomarker names")

##### Log transformed

In [ ]:
plot_corr_matrix(all_cell_log_means, "Cell-aggregated raw correlation matrix (log transformed)", "log_cell_corr")

In [ ]:


# Compare with the original linear correlation matrix from cell 44
if 'pixel_corr_matrix' in globals():
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    sns.heatmap(cell_corr_matrix, cmap="coolwarm", center=0)
    plt.title("Linear Scale Correlations")
    
    plt.subplot(1, 2, 2)
    sns.heatmap(cell_log_corr_matrix, cmap="coolwarm", center=0)
    plt.title("Log-transformed Correlations")
    
    plt.show()

#### Non-Cell Level

In [ ]:
with h5py.File("data.h5", "r+") as hf:
    vol_grp = hf["volumes"]
    data = vol_grp['data']
    training_idx = vol_grp["train_idx"]
    
    # Get cell masks to identify empty regions
    cell_mask_grp = hf["cell_masks"]
    cell_masks = cell_mask_grp["data"]
    
    N, H, W = cell_masks.shape
    
    # Create non_cell_masks group if it doesn't exist
    if "non_cell_masks" in hf:
        del hf["non_cell_masks"]
    
    non_cell_mask_grp = hf.create_group("non_cell_masks")
    non_cell_masks = non_cell_mask_grp.create_dataset("data", shape=(N, H, W), dtype=cell_masks.dtype)
    
    # Process each image
    for img_idx in tqdm(range(N), desc="Creating non-cell masks"):
        cell_mask = cell_masks[img_idx]
        non_cell_mask = np.zeros_like(cell_mask)
        
        square_id = 1
        
        # Scan image in 5x5 grid pattern
        for y in range(0, H-4, 5):  # Step by 5 to avoid overlap
            for x in range(0, W-4, 5):  # Step by 5 to avoid overlap
                # Check if entire 5x5 square has no cells (all zeros)
                square_region = cell_mask[y:y+5, x:x+5]
                
                if np.all(square_region == 0):
                    # Assign unique ID to this 5x5 square
                    non_cell_mask[y:y+5, x:x+5] = square_id
                    square_id += 1
        
        non_cell_masks[img_idx] = non_cell_mask
        
        if img_idx == 0:  # Print stats for first image
            unique_ids = np.unique(non_cell_mask)
            print(f"Created {len(unique_ids)-1} non-cell regions in first image")

In [ ]:
number_of_super_pixels_per_image = 400
all_non_cell_means = []
all_non_cell_log_means = []
non_cell_ids = []
with h5py.File("data.h5", "r") as hf:
    vol_grp = hf["volumes"]
    data = vol_grp['data']  
    training_idx = vol_grp["train_idx"]
    # N*H*W*C

    case_ids = vol_grp["case_ids"]

    non_cell_mask_grp = hf["non_cell_masks"]
    non_cell_masks = non_cell_mask_grp["data"]
    # N*H*W
    
    # Get dimensions
    _, H, W, C = data.shape

    for train_idx in tqdm(range(min(len(training_idx),NUMBER_OF_IMAGES)), position=0):

        img_idx = training_idx[train_idx]


        # Get the current image slice and its non_cell mask
        img_data = data[img_idx]  # Shape: H*W*C
        img_mask = non_cell_masks[img_idx]  # Shape: H*W
        case_id = case_ids[img_idx]
        
        # Find unique non_cell IDs in this image (excluding 0 for background)
        img_non_cell_ids = np.unique(img_mask)
        img_non_cell_ids = img_non_cell_ids[img_non_cell_ids > 0]
        samples = np.random.choice(img_non_cell_ids, size=min(number_of_super_pixels_per_image, len(img_non_cell_ids)), replace=False)

        for i, non_cell_id in enumerate(samples):
            print(f"processing non_cell {i}", end="\r")
            # Create mask for this specific non_cell in this specific image
            non_cell_mask = (img_mask == non_cell_id)  # Shape: H*W

            
            # Sum up biomarker values for this non_cell
            non_cell_means = np.zeros(C)
            non_cell_log_means = np.zeros(C)
            for c in range(C):
                # Apply mask to this channel and calculate mean
                non_cell_pixels = img_data[:, :, c][non_cell_mask]
                non_cell_log_pixels = np.log1p(img_data[:, :, c][non_cell_mask])
                non_cell_means[c] = np.mean(non_cell_pixels) if non_cell_pixels.size > 0 else 0
                non_cell_log_means[c] = np.mean(non_cell_log_pixels) if non_cell_log_pixels.size > 0 else 0
            
            all_non_cell_means.append(non_cell_means)
            all_non_cell_log_means.append(non_cell_log_means)
            non_cell_ids.append((case_id, non_cell_id))
        print()
            

In [ ]:

all_non_cell_means = np.array(all_non_cell_means)
all_non_cell_log_means = np.array(all_non_cell_log_means)

In [ ]:

non_cell_corr_matrix = plot_corr_matrix(all_non_cell_means, "non-segmented corr matrix", "lin_non_cell_corr")

In [ ]:

non_cell_log_corr_matrix = plot_corr_matrix(all_non_cell_log_means, "non-segmented corr matrix (log transformed)", "log_non_cell_corr")

In [ ]:
len(all_non_cell_means)

In [ ]:
plot_peripherin_correlations(non_cell_corr_matrix, "Unsegmented super-pixel correlations", "filtered_peripherin_correlations")

In [ ]:

plot_peripherin_correlations(non_cell_log_corr_matrix, "Unsegmented super-pixel correlations(log transformed)", "log_filtered_peripherin_correlations")

In [ ]:

# Find the index of Peripherin in biomarker_names
peripherin_idx = None
for i, name in enumerate(biomarker_names):
    if 'Neurofilament' in name:
        peripherin_idx = i
        break

if peripherin_idx is not None:
    # Get peripherin values for all non_cells
    peripherin_values = np.array(all_non_cell_means)[:, peripherin_idx]
    
    # Count positive values (greater than 0)
    positive_count = np.sum(peripherin_values > 0)
    total_count = len(peripherin_values)
    
    print(f"Peripherin positive non_cells: {positive_count}")
    print(f"Total non_cells: {total_count}")
    print(f"Percentage positive: {positive_count/total_count*100:.2f}%")
    
    # Show distribution of peripherin values
    print(f"Peripherin value range: {peripherin_values.min():.3f} - {peripherin_values.max():.3f}")
    print(f"Mean peripherin value: {peripherin_values.mean():.3f}")
    print(f"Median peripherin value: {np.median(peripherin_values):.3f}")
else:
    print("Peripherin not found in biomarker names")

In [ ]:
# Compute correlation matrix (pixel-wise, across channels)
peripherin_idx = 31  # Peripherin index from biomarker_names
peripherin_values = np.array(all_non_cell_log_means)[:, peripherin_idx]
positive_mask = peripherin_values > 0

all_positive_peripherin_non_cell_means = np.array(all_non_cell_log_means)[positive_mask]
plot_corr_matrix(all_positive_peripherin_non_cell_means, "positive peripherine filtered means", "lin_pos_peripherine_filtered_corr")

### Spearman Correlation

In [ ]:
rho, p = spearmanr(all_non_cell_means, axis=0)      # rho: (40, 40) incl. 2 summary rows/cols
spearman_corr = rho[:38, :38]    


non_cell_rho, non_cell_p = spearmanr(all_positive_peripherin_non_cell_means, axis=0)      # rho: (40, 40) incl. 2 summary rows/cols
non_cell_spearman_corr = non_cell_rho[:38, :38]    


In [ ]:


fig, axs = plt.subplots(1, 2, figsize=(22, 10))

# Pixel-level MI matrix
sns.heatmap(spearman_corr, 
            xticklabels=biomarker_names, 
            yticklabels=biomarker_names,
            cmap="viridis", annot=False, ax=axs[0])
axs[0].set_title("Pairwise Spearman Correlation (cell Level)")
axs[0].tick_params(axis='x', rotation=90)

# Cell-level MI matrix
sns.heatmap(non_cell_spearman_corr, 
            xticklabels=biomarker_names, 
            yticklabels=biomarker_names,
            cmap="viridis", annot=False, ax=axs[1])
axs[1].set_title("Pairwise Spearman Correlation (non Cell Means)")
axs[1].tick_params(axis='x', rotation=90)

plt.tight_layout()
plt.show()

In [ ]:
plot_peripherin_correlations(non_cell_spearman_corr, "spearman corr", "peripherin_spearmans")

### Pairwise Mutual Information (MI) Between Biomarkers

Mutual information (MI) quantifies the dependency between two variables, capturing both linear and non-linear relationships. Here, we compute the pairwise MI between all biomarker channels using pixel-level data, and visualize the MI matrix as a heatmap. This provides an alternative perspective to correlation analysis for understanding biomarker relationships.

In [ ]:
len(all_non_cell_log_means)

In [ ]:
biomarkers_to_investigate = list(range(38))

n_mi_pixels = 10_000
mi_pixels = all_non_cell_log_means[np.random.choice(len(all_non_cell_log_means), size=n_mi_pixels, replace=False)]
# mi_pixels = log_pixels

# Compute pairwise mutual information matrix
C = len(biomarkers_to_investigate)
mi_matrix = np.zeros((C, C))
for i in tqdm(range(C), "Outer loop", position=0):
    for j in tqdm(range(C), "Inner loop", position=1, leave=False):
        idx1 = biomarkers_to_investigate[i]
        idx2 = biomarkers_to_investigate[j]
        if i == j:
            mi_matrix[i, j] = np.nan  # MI with self is not informative
        else:
            # MI is not symmetric, so average MI(X|Y) and MI(Y|X)
            mi_ij = mutual_info_regression(mi_pixels[:, [idx1]], mi_pixels[:, idx2], random_state=42)
            mi_ji = mutual_info_regression(mi_pixels[:, [idx2]], mi_pixels[:, idx1], random_state=42)
            mi_matrix[i, j] = 0.5 * (mi_ij[0] + mi_ji[0])



In [ ]:
mi_matrix_no_nan = np.nan_to_num(mi_matrix)
plot_corr_matrix(mi_matrix_no_nan, "mutual information","mutual_information" )

In [ ]:
plot_peripherin_correlations(mi_matrix_no_nan, "peripherin mutual info", "peripherin_mi")


### Basic Predictive Modelling

In [ ]:
def bpm(dataset, biomarkers_to_investigate):
    # Ensure reproducibility within this function
    np.random.seed(42)
    
    C = len(biomarkers_to_investigate)
    df_pixels = pd.DataFrame(
        dataset[:, biomarkers_to_investigate],
        columns=[biomarker_names[i] for i in biomarkers_to_investigate]
    )
    def make(reg):
        """Wrap regressor in X-scaler pipeline; optionally y-scaler."""
        pipe = Pipeline([('scaler', StandardScaler()), ('reg', reg)])
        return TransformedTargetRegressor(
                    regressor=pipe,
                    transformer=StandardScaler())
        return pipe

    results = []

    # --- Define models to try ---
    models = {
        "MeanBaseline": DummyRegressor(strategy='mean'),   
        "MedianBaseline": DummyRegressor(strategy='median'),   
        "LinearRegression": make(LinearRegression()),
        "Ridge": make(Ridge()),
        "RandomForest": RandomForestRegressor(n_estimators=20, max_depth=5, random_state=42, n_jobs=-1)
    }

    # --- Predict each biomarker from all others ---
    for target_name in tqdm(df_pixels.columns, desc="Targets"):
        X = df_pixels.drop(columns=target_name)
        y = df_pixels[target_name]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        for model_name, model in tqdm(models.items(), desc=f"Models for {target_name}", leave=False):
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            r2 = r2_score(y_test, y_pred)
            nmse = mean_squared_error(y_test, y_pred)
            results.append({
                "Target": target_name,
                "Model": model_name,
                "R2": r2,
                "NMSE": nmse
            })

    results_df = pd.DataFrame(results)
    biomarker_meta = load_biomarkers()
    metals_repeated = np.repeat(biomarker_meta["Metal"].values, len(models))
    results_df["Metal"] = metals_repeated[:len(results_df)]

    results_df['target_unique'] = results_df.groupby('Target').cumcount()//len(models) + 1
    results_df['target_unique'] = results_df.apply(lambda x: f"{x['Target']} {x['target_unique']}" if (results_df['Target'] == x['Target']).sum()//len(models) > 1 else x['Target'], axis=1)

    return models, results_df


#### Non-Cells

In [ ]:
len(all_non_cell_log_means)

In [ ]:
# Ensure reproducibility for this analysis
np.random.seed(42)

biomarkers_to_investigate = list(range(len(biomarker_names)))  # Predict all biomarkers
dataset = np.random.permutation(all_non_cell_log_means)

filtered_models, filtered_results = bpm(dataset, biomarkers_to_investigate)

In [ ]:

filtered_plot_df = filtered_results.melt(
              id_vars=['target_unique', 'Model'],
              value_vars=['R2', 'NMSE'],        # which metrics to plot
              var_name='Metric',
              value_name='Score')

In [ ]:
filtered_plot_df

In [ ]:
custom_palette = {
    "MeanBaseline": "#AA4FC8",  # red
    "LinearRegression": "#3773C2",  # blue
    "Ridge":"#FF6B6B" ,  # purple
    "RandomForest": "#F0B400",  # orange–gold
}

# Filter out RandomForest from the plot data
filtered_plot_df_no_median = filtered_plot_df[
    filtered_plot_df["Model"] != "MedianBaseline"
]

g = sns.catplot(
    data=filtered_plot_df_no_median,
    kind="bar",
    x="target_unique",
    y="Score",
    hue="Model",
    row="Metric",  # one subplot for R2, one for NMSE
    sharex=False,
    sharey=False,  # independent y-axes
    height=5,
    aspect=2,
    legend_out=False,
    palette=custom_palette,  # Apply the custom palette
)

g.figure.suptitle(
    "Predictive Performance per Biomarker (filtered Level)", y=1.05, fontsize=14
)

g.set_xticklabels(biomarker_names, rotation=90, ha="right", fontsize=8)
# 4 ─ add a bit of breathing room below the labels
g.figure.subplots_adjust(bottom=0.25, hspace=0.35)
g.axes[0][0].set_ylabel("R\u00b2")  # top panel
g.axes[1][0].set_ylabel("nMSE")  # bottom panel
g.set_titles("{row_name}")  # row titles: R2 / NMSE

plt.savefig("plots/bnp_performance.pdf")
plt.show()

In [ ]:
# Filter data for R2 only and exclude MedianBaseline
r2_data = filtered_results[(filtered_results["Model"] != "MedianBaseline")].copy()

# Create horizontal grouped bar plot optimized for LaTeX
fig, ax = plt.subplots(figsize=(10, 12))  # Taller for horizontal layout

# Get unique biomarkers and models
biomarkers = r2_data["target_unique"].unique()
models = ["MeanBaseline", "LinearRegression", "Ridge", "RandomForest"]
models_filtered = [m for m in models if m in r2_data["Model"].unique()]

# Sort biomarkers by best RandomForest performance for better visual hierarchy
rf_scores = {}
for bio in biomarkers:
    rf_score = r2_data[
        (r2_data["target_unique"] == bio) & (r2_data["Model"] == "RandomForest")
    ]["R2"].iloc[0]
    rf_scores[bio] = rf_score

biomarkers_sorted = sorted(
    biomarkers, key=lambda x: rf_scores[x], reverse=False
)  # Changed to reverse=False for increasing order

# Set up bar positions with optimal spacing
y_pos = np.arange(len(biomarkers_sorted))
bar_height = 0.18  # Bar height for horizontal bars
offset = np.linspace(-1.5 * bar_height, 1.5 * bar_height, len(models_filtered))

# Color palette for models
model_colors = {
    "MeanBaseline": "#8B949E",  # Gray for baseline
    "LinearRegression": "#1F77B4",  # Blue
    "Ridge": "#FF7F0E",  # Orange
    "RandomForest": "#2CA02C",  # Green for best performer
}

# Create horizontal bars for each model
for i, model in enumerate(models_filtered):
    model_data = r2_data[r2_data["Model"] == model]
    scores = [
        model_data[model_data["target_unique"] == bio]["R2"].iloc[0]
        for bio in biomarkers_sorted
    ]

    # Add error handling for missing data
    scores = [max(0, score) for score in scores]  # Ensure no negative values

    # Color-code bars based on whether biomarker is nerve-related
    bar_colors = []
    for bio in biomarkers_sorted:
        if bio in nerve_biomarkers:
            # Use darker/saturated version of model color for nerve biomarkers
            base_color = model_colors[model]
            if model == "RandomForest":
                bar_colors.append("#1A7A1A")  # Darker green
            elif model == "Ridge":
                bar_colors.append("#CC5500")  # Darker orange
            elif model == "LinearRegression":
                bar_colors.append("#0F4C75")  # Darker blue
            else:
                bar_colors.append("#5C6268")  # Darker gray
        else:
            bar_colors.append(model_colors[model])

    bars = ax.barh(
        y_pos + offset[i],
        scores,
        bar_height,
        color=bar_colors,
        alpha=0.8,
        edgecolor="white",
        linewidth=0.5,
    )

# Add model labels to legend
from matplotlib.patches import Patch

legend_elements = []
for model in models_filtered:
    legend_elements.append(Patch(facecolor=model_colors[model], label=model))

# Add nerve biomarker indicator to legend
legend_elements.append(
    Patch(facecolor="#000000", label="Nerve Biomarkers (darker shades)")
)

# Customize plot for LaTeX
ax.set_ylabel(
    "Target Biomarkers (sorted by Random Forest performance)",
    fontsize=11,
    fontweight="bold",
)
ax.set_xlabel("R² Score", fontsize=11, fontweight="bold")
ax.set_title(
    "Cross-Validation Performance: Predicting Each Biomarker from Others",
    fontsize=11,
    fontweight="bold",
    pad=15,
)

# Position y-ticks with nerve biomarkers highlighted
tick_labels = []
for bio in biomarkers_sorted:
    if bio in nerve_biomarkers:
        tick_labels.append(f"$\\mathbf{{{bio}}}$")  # Bold for nerve biomarkers
    else:
        tick_labels.append(bio)

ax.set_yticks(y_pos)
ax.set_yticklabels(tick_labels, fontsize=10)
ax.tick_params(axis="x", labelsize=9)

# Enhanced legend with better positioning
ax.legend(
    handles=legend_elements,
    loc="lower right",
    fontsize=10,
    title_fontsize=11,
    title="Legend",
    frameon=True,
    fancybox=False,
    shadow=False,
)

# Subtle grid only on x-axis
ax.grid(axis="x", alpha=0.3, linewidth=0.5, linestyle="-")
ax.set_axisbelow(True)

# Set x-axis limits with some headroom
max_score = max(
    [max(r2_data[r2_data["Model"] == model]["R2"]) for model in models_filtered]
)
ax.set_xlim(0, min(1.0, max_score * 1.15))  # Cap at 1.0 since R² shouldn't exceed 1

# Tight layout optimized for LaTeX
plt.tight_layout(pad=1.0)

# Save with high quality for LaTeX
plt.savefig(
    "plots/grouped_r2_performance_horizontal.pdf",
    dpi=300,
    bbox_inches="tight",
    facecolor="white",
    edgecolor="none",
    format="pdf",
)
plt.show()

# Print summary statistics including nerve biomarkers
nerve_biomarkers_in_data = [bio for bio in biomarkers_sorted if bio in nerve_biomarkers]
other_biomarkers_in_data = [
    bio for bio in biomarkers_sorted if bio not in nerve_biomarkers
]

print(f"\nSummary Statistics:")
print(
    f"Best performing target (RandomForest): {biomarkers_sorted[-1]} (R² = {rf_scores[biomarkers_sorted[-1]]:.3f})"
)  # Changed to [-1] for last element
print(
    f"Worst performing target (RandomForest): {biomarkers_sorted[0]} (R² = {rf_scores[biomarkers_sorted[0]]:.3f})"
)  # Changed to [0] for first element
print(
    f"Number of biomarkers with R² > 0.5: {sum(1 for score in rf_scores.values() if score > 0.5)}"
)
print(
    f"Number of biomarkers with R² > 0.3: {sum(1 for score in rf_scores.values() if score > 0.3)}"
)
print(f"\nNerve biomarkers performance:")
for bio in nerve_biomarkers_in_data:
    print(f"  {bio}: R² = {rf_scores[bio]:.3f}")
print(
    f"Average R² for nerve biomarkers: {np.mean([rf_scores[bio] for bio in nerve_biomarkers_in_data]):.3f}"
)
print(
    f"Average R² for other biomarkers: {np.mean([rf_scores[bio] for bio in other_biomarkers_in_data]):.3f}"
)

In [ ]:
from matplotlib.patches import Patch

def plot_top_correlations_grid(correlation_matrices, titles, filename="top_correlations_grid", top_n=5):
    """
    Create a grid of bar plots showing top N correlations with Peripherin for different correlation matrices.
    
    Parameters:
    correlation_matrices: list of correlation matrices
    titles: list of titles for each subplot
    filename: base filename for saving the plot
    top_n: number of top correlations to show (default: 5)
    """
    # Find the index of Peripherin in biomarker_names
    peripherin_idx = None
    for i, name in enumerate(biomarker_names):
        if 'Peripherin' in name:
            peripherin_idx = i
            break
    
    if peripherin_idx is None:
        print("Peripherin not found in biomarker names")
        return
    
    n_plots = len(correlation_matrices)
    n_cols = 2
    n_rows = (n_plots + 1) // 2
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4*n_rows))
    if n_rows == 1:
        axes = axes.reshape(1, -1)
    
    for idx, (corr_matrix, title) in enumerate(zip(correlation_matrices, titles)):
        row = idx // n_cols
        col = idx % n_cols
        ax = axes[row, col]
        
        # Extract correlations between Peripherin and all other biomarkers
        peripherin_correlations = corr_matrix[peripherin_idx, :]
        
        # Remove Peripherin's self-correlation
        plot_correlations = np.delete(peripherin_correlations, peripherin_idx)
        plot_biomarker_names = [name for i, name in enumerate(biomarker_names) if i != peripherin_idx]
        
        # Get top N correlations
        top_indices = np.argsort(plot_correlations)[-top_n:][::-1]  # Top N, highest first
        top_correlations = plot_correlations[top_indices]
        top_names = [plot_biomarker_names[i] for i in top_indices]
        
        # Color based on nerve biomarkers
        colors = []
        for name in top_names:
            if name in nerve_biomarkers:
                colors.append('#FF6B6B')  # Red for nerve biomarkers
            else:
                colors.append('#4ECDC4')  # Teal for other biomarkers
        
        # Create bar plot
        bars = ax.bar(range(len(top_names)), top_correlations, color=colors, 
                     alpha=0.8, edgecolor='black', linewidth=0.5)
        
        ax.set_title(title, fontsize=12, fontweight='bold', pad=10)
        ax.set_ylabel('Score with peripherin', fontsize=12)
        ax.set_xticks(range(len(top_names)))
        ax.set_xticklabels(top_names, rotation=45, ha='right', fontsize=12)
        
        # Add value labels on bars
        for bar, value in zip(bars, top_correlations):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.005,
                   f'{value:.3f}', ha='center', va='bottom', fontsize=12)
        
        ax.grid(axis='y', alpha=0.3, linewidth=0.5)
        ax.set_ylim(0, max(top_correlations) * 1.15)
    
    # Hide empty subplots if odd number of plots
    if n_plots % 2 == 1:
        axes[-1, -1].set_visible(False)
    
    # Add legend
    legend_elements = [
        Patch(facecolor='#FF6B6B', label='Nerve Biomarkers'),
        Patch(facecolor='#4ECDC4', label='Other Biomarkers')
    ]
    fig.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))
    
    plt.tight_layout()
    plt.savefig(f"plots/{filename}.pdf", dpi=300, bbox_inches='tight', 
               facecolor='white', edgecolor='none')
    plt.show()

# Prepare the correlation matrices and titles
correlation_matrices = [
    pix_corr,
    log_corr, 
    non_cell_corr_matrix,
    non_cell_log_corr_matrix,
    non_cell_spearman_corr,
    mi_matrix_no_nan
]

titles = [
    "Pixel-wise correlations (Linear)",
    "Pixel-wise correlations (Log Transformed)",
    "Non-segmented correlations (Linear)",
    "Non-segmented correlations (Log Transformed)",
    "Non-segmented Spearman (Log Transformed)",
    "Non-segmented Mutual information (Log Transformed)"
]

# Example usage - you can now specify different values for top_n
plot_top_correlations_grid(correlation_matrices, titles, "peripherin_top10_grid", top_n=10)
# plot_top_correlations_grid(correlation_matrices, titles, "peripherin_top10_grid", top_n=10)

In [ ]:
# Filter data for nerve biomarkers and specific models
nerve_data = r2_data[
    (r2_data["target_unique"].isin(nerve_biomarkers)) & 
    (r2_data["Model"].isin(["Ridge", "RandomForest"]))
].copy()

# Create comparison plot
fig, ax = plt.subplots(figsize=(12, 8))

# Get unique nerve biomarkers in the data
nerve_biomarkers_in_filtered = [bio for bio in nerve_biomarkers if bio in nerve_data["target_unique"].unique()]

# Set up positions for grouped bars
x_pos = np.arange(len(nerve_biomarkers_in_filtered))
width = 0.35

# Get Ridge and RandomForest scores for each nerve biomarker
ridge_scores = []
rf_scores = []

for bio in nerve_biomarkers_in_filtered:
    ridge_score = nerve_data[
        (nerve_data["target_unique"] == bio) & (nerve_data["Model"] == "Ridge")
    ]["R2"].iloc[0]
    rf_score = nerve_data[
        (nerve_data["target_unique"] == bio) & (nerve_data["Model"] == "RandomForest")
    ]["R2"].iloc[0]
    
    ridge_scores.append(ridge_score)
    rf_scores.append(rf_score)

# Create grouped bar plot
bars1 = ax.bar(x_pos - width/2, ridge_scores, width, label='Ridge', 
               color='#FF7F0E', alpha=0.8, edgecolor='black', linewidth=0.5)
bars2 = ax.bar(x_pos + width/2, rf_scores, width, label='RandomForest', 
               color='#2CA02C', alpha=0.8, edgecolor='black', linewidth=0.5)

# Add value labels on bars
for bar, score in zip(bars1, ridge_scores):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
           f'{score:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

for bar, score in zip(bars2, rf_scores):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
           f'{score:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

# Customize the plot
ax.set_xlabel('Nerve Biomarkers', fontsize=12, fontweight='bold')
ax.set_ylabel('R² Score', fontsize=12, fontweight='bold')
ax.set_title('Model Performance Comparison: Ridge vs Random Forest\nfor Nerve-Related Biomarkers', 
             fontsize=14, fontweight='bold', pad=20)

ax.set_xticks(x_pos)
ax.set_xticklabels(nerve_biomarkers_in_filtered, rotation=45, ha='right', fontsize=11)
ax.tick_params(axis='y', labelsize=10)

# Add legend
ax.legend(loc='upper left', fontsize=12, title='Model', title_fontsize=12)

# Add grid for better readability
ax.grid(axis='y', alpha=0.3, linewidth=0.5)
ax.set_axisbelow(True)

# Set y-axis limits
ax.set_ylim(0, max(max(ridge_scores), max(rf_scores)) * 1.15)

# Tight layout
plt.tight_layout()

# Save the plot
plt.savefig("plots/nerve_biomarkers_model_comparison.pdf", dpi=300, bbox_inches='tight', 
           facecolor='white', edgecolor='none')
plt.show()

# Print comparison statistics
print("Nerve Biomarkers Performance Comparison:")
print("=" * 50)
for i, bio in enumerate(nerve_biomarkers_in_filtered):
    ridge_score = ridge_scores[i]
    rf_score = rf_scores[i]
    improvement = rf_score - ridge_score
    print(f"{bio:15}: Ridge={ridge_score:.3f}, RF={rf_score:.3f}, Improvement={improvement:+.3f}")

print(f"\nAverage Ridge R²: {np.mean(ridge_scores):.3f}")
print(f"Average RandomForest R²: {np.mean(rf_scores):.3f}")
print(f"Average Improvement: {np.mean(rf_scores) - np.mean(ridge_scores):+.3f}")